In [282]:
import packages.initialization
import packages.pioneer3dx as p3dx
p3dx.init()
import cv2
import numpy
import matplotlib.pyplot as plt
%matplotlib inline

Found ROS controller /pioneer3dx_20756_ip_172_31_36_85


In [283]:
def blue_balls():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower_blue = numpy.array([105,  30,  30])
    upper_blue = numpy.array([130, 255, 255])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = 4
    detector = cv2.SimpleBlobDetector(params)
    reversemask = 255-mask
    keypoints = detector.detect(reversemask)
    l = []
    for i,kp in enumerate(keypoints):
        l.append((kp.size,) + kp.pt)
    l.sort(key=lambda tup: tup[0], reverse=True)
    return l

In [284]:
def Detect_Ball():
    b = blue_balls()
    while len(b) == 0 or not is_blob_centered():
        p3dx.move(-0.5,0.5)
        b = blue_balls()
    p3dx.move(0,0) 
    #while not is_blob_centered():
         #p3dx.move(-0.5,0.5)
    #p3dx.stop()

def parameters():
    b = blue_balls()
    if len(b) != 0:
        Dia = b[0][0]
        cx = b[0][1]
        cy = b[0][2] 
    else:
        Dia = 0.1
        cx = None
        cy = 110
    return Dia, cx, cy

def is_blob_centered():
    Dia, cx, cy = parameters()
    if Dia > 0 and cx >= 70 and cx < 75:
        return True
    else:
        return False

def is_blob_close():
    Dia, cx, cy = parameters()
    if Dia > 0 and cy > 95:
        return True
    else:
        return False
    
def Approach_Ball():
    while not is_blob_close():
        p3dx.move(1.5,1.5)
    p3dx.stop() 
    
def Pick():
    #p3dx.tilt(-0.47)
    #p3dx.sleep(0.3)
    #while not is_blob_close():
    p3dx.move(1.30,1.30)
    p3dx.sleep(1.30)
    p3dx.move(0,0)
    p3dx.gripper(0.05,0.0)  # down and closed fingers
    p3dx.sleep(1)
    p3dx.gripper(-0.05,0.0)   # up and closed fingers
    p3dx.sleep(1)

    
def is_red_blob_close():
    area, cx, cy = red_color_blob()
    if area > 20 and cy >= 91:
        return True
    else:
        return False  
    
def red_color_blob():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower_red = numpy.array([ 0, 50, 50])   # red
    upper_red = numpy.array([ 10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    mask[0:70,0:150] = 0
    M = cv2.moments(mask)
    area = M['m00']
    if area > 0:
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
    else:
        cx = None
        cy = None
    return area, cx, cy

def is_red_detected():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower_red = numpy.array([ 0, 100, 100])   # red
    upper_red = numpy.array([ 10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    M = cv2.moments(mask)
    area = M['m00']
    if area > 0: 
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
        if area > 10000  and cx >= 50:
            return True
        else:
            return False
def Approach_area():    
    while not is_red_blob_close():
         p3dx.move(1.0,1.0)
    p3dx.move(0,0)
    #p3dx.tilt(-0.47)
    #p3dx.sleep(0.5)
    #while not is_red_blob_close():
        #p3dx.move(1.0,1.0)
        #p3dx.sleep(0.5)
    #p3dx.move(0,0)        

def Search_destination_area():
    p3dx.tilt(-0.35)
    p3dx.sleep(1)
    while not is_red_detected():
          p3dx.move(-0.5,0.5)
    p3dx.move(0,0) 
    
    
def Release_ball():
    p3dx.move(1.50,1.50)
    p3dx.sleep(1.5)
    p3dx.move(0,0) 
    p3dx.gripper(0.05,0.10)  # down and open fingers
    p3dx.sleep(0.7)
    p3dx.move(-2.50,-2.50)
    p3dx.sleep(3.0)
    p3dx.move(-1.50,1.50)
    p3dx.sleep(1.0)
    p3dx.move(0,0)
        

In [285]:
try:
    while True:
        p3dx.tilt(-0.35)
        p3dx.sleep(0.5)
        p3dx.gripper(0.05,0.1)  # down and open fingers
        p3dx.sleep(0.5)
        Detect_Ball()
        p3dx.sleep(0.25)
        Approach_Ball()
        p3dx.sleep(0.25)
        Pick()
        Search_destination_area()
        Approach_area()
        Release_ball()
except KeyboardInterrupt:
    move(0,0)

Exception: The robot needs initialization - has the simulation been restarted?